# Wine Quality:
<hr style="border:2px solid black">

## Goal: 
- Discover independence of features within data
- Utilizing said features to develop machine learning models that will determine the quality of wine 
- Determine the drivers of quality
- The insights discovered will be used to estimate quality and provide a more robust understanding of the attributes of a the wine and it's associated quality
<hr style="border:2px solid black">

# Imports

In [2]:
# Standard ds imports:
import pandas as pd
import numpy as np

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

# Statisitics
from scipy import stats

import os
import acquire as a
import wrangle as w

<hr style="border:2px solid black">

# **Acquire**
Plan --> **Acquire** --> Prepare --> Explore --> Model --> Deliver

In [3]:
df = a.acquire_wine()

In [4]:
# Acquire
if os.path.exists('winequality_red.csv'):
    red_df = pd.read_csv('winequality_red.csv')

if os.path.exists('winequality_white.csv'):
    white_df = pd.read_csv('winequality_white.csv')

In [5]:
red_df['is_red'] = 1
white_df['is_red'] = 0

In [6]:
df = pd.concat([white_df, red_df], axis=0, ignore_index=True)

<hr style="border:2px solid black">

# **Prepare**

In [7]:
# Splitting data
train, validate, test = w.splitter(df)

Dataframe: (6497, 13) 100%
Train: (4157, 13) | ~60%
Validate: (1040, 13) | ~20%
Test: (1300, 13) | ~20%


In [ ]:
# Scaling of x_train, val, test // scaler TBD

# x_train_scaled, x_val_scaled, x_test_scaled = w.QuickScale()  

<hr style="border:2px solid black">

# **Explore**
Plan --> Acquire --> Prepare --> **Explore** --> Model --> Deliver

<hr style="border:2px solid black">

# **Modeling**
Plan --> Acquire --> Prepare --> Explore --> **Model** --> Deliver

<hr style="border:2px solid black">

# **Conclusions**
